## Model Development

#### import data

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
df = pd.read_csv('data/df_fs.csv', sep=",", index_col=0)

In [9]:
df.head()

,date,Open,Low,Close,Volume,30_day_MA,60_day_MA,90_day_MA,RSI,MACD,MACD_Histogram,Price_Change_Pct,Article Length,article_sentiment,Price_Change_Category,smoothed_sentiment
0,2016-07-20,56.84,55.53,55.91,89893300.00,51.95,52.31,50.52,71.72,0.56,0.52,5.31,1394.00,4.00,positive,4.00
1,2016-07-25,56.74,56.26,56.73,25610600.00,52.10,52.40,50.66,74.61,0.84,0.64,0.28,623.00,4.00,positive,4.00
2,2016-08-01,56.75,56.14,56.58,26003400.00,52.25,52.48,50.80,91.39,1.03,0.67,-0.18,269.00,4.00,negative,4.00
3,2016-08-03,57.11,56.49,56.97,22075600.00,52.41,52.55,50.95,91.80,1.20,0.67,0.69,904.00,4.00,positive,4.00
4,2016-08-05,58.21,57.45,57.96,29335200.00,52.61,52.66,51.12,92.48,1.40,0.70,0.99,679.00,1.00,positive,3.40


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4983 entries, 0 to 4982
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   4983 non-null   object 
 1   Open                   4983 non-null   float64
 2   Low                    4983 non-null   float64
 3   Close                  4983 non-null   float64
 4   Volume                 4983 non-null   float64
 5   30_day_MA              4983 non-null   float64
 6   60_day_MA              4983 non-null   float64
 7   90_day_MA              4983 non-null   float64
 8   RSI                    4983 non-null   float64
 9   MACD                   4983 non-null   float64
 10  MACD_Histogram         4983 non-null   float64
 11  Price_Change_Pct       4983 non-null   float64
 12  Article Length         4983 non-null   float64
 13  article_sentiment      4983 non-null   float64
 14  Price_Change_Category  4983 non-null   object 
 15  smoothed_

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# Define relevant features and the target variable

#additional_features = ["MACD_Histogram","RSI","MACD", 'EMA_12',"Article Length" ]
#additional_features = ['RSI', 'Volume', 'MACD_Histogram', 'Timestamp', 'Day', 'Signal_Line', 'Month', 'DayOfWeek', 'Close', 'Article Length', 'Open', 'Low']
#X = df[['article_sentiment'] + additional_features]  # Include 'article_sentiment' and other features

#X = df.drop(["Price_Change_Pct", "date","Price_Change_Category"], axis=1)  # Exclude target variable and date
X = df[['Open', 'Low', 'Close', '60_day_MA', 'RSI', 'MACD', 'MACD_Histogram', 'article_sentiment']]
y = df['Price_Change_Pct']  # Target variable

# Split the data into training and testing sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create a linear regression model
lin_reg = LinearRegression()

# Fit the model to the training data
lin_reg.fit(X_train_scaled, y_train)

# Predict on the test data
y_pred = lin_reg.predict(X_test_scaled)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Output model coefficients and performance metrics
coefficients = dict(zip(X.columns, lin_reg.coef_))  # Get model coefficients
print("Evaluation Metrics:")
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2):", r2)

# Add a constant term to the features
X_train_with_const = sm.add_constant(X_train_scaled)

# Create and fit the model
model = sm.OLS(y_train, X_train_with_const)
results = model.fit()

# Print the summary
print("\nLinear Regression Summary:")
print(results.summary())


Evaluation Metrics:
Mean Absolute Error (MAE): 0.7903134051632317
Mean Squared Error (MSE): 1.6012902408812697
R-squared (R2): 0.6442192078734159

Linear Regression Summary:
                            OLS Regression Results                            
Dep. Variable:       Price_Change_Pct   R-squared:                       0.689
Model:                            OLS   Adj. R-squared:                  0.688
Method:                 Least Squares   F-statistic:                     1102.
Date:                Thu, 25 Apr 2024   Prob (F-statistic):               0.00
Time:                        11:41:33   Log-Likelihood:                -6022.3
No. Observations:                3986   AIC:                         1.206e+04
Df Residuals:                    3977   BIC:                         1.212e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err    

### XGBRegressor 

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor  # Import XGBRegressor

# Define features and the target variable
X = df[['Open', 'Low', 'Close', '60_day_MA', 'RSI', 'MACD', 'MACD_Histogram', 'article_sentiment']]
y = df['Price_Change_Pct']  # Target variable

# Split the data into training and testing sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Standardize the training set
X_test_scaled = scaler.transform(X_test)  # Standardize the testing set

# Create the XGBRegressor model
xgb_reg = XGBRegressor(objective='reg:squarederror', random_state=42)  # 'reg:squarederror' for regression tasks

# Fit the model to the training data
xgb_reg.fit(X_train_scaled, y_train)  # Ensure the model is trained

# Predict on the test data
y_pred = xgb_reg.predict(X_test_scaled)  # Predict on the test set

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)  # Mean Absolute Error
mse = mean_squared_error(y_test, y_pred)  # Mean Squared Error
r2 = r2_score(y_test, y_pred)  # R-squared

# Retrieve feature importance using the correct method
feature_importance = xgb_reg.get_booster().get_score(importance_type='weight')  # Get feature importance by weight

# Output model coefficients and performance metrics
print("Evaluation Metrics:")
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2):", r2)

print("\nFeature Importance:")
print(feature_importance)  # Display feature importance


Evaluation Metrics:
Mean Absolute Error (MAE): 0.5094086234515847
Mean Squared Error (MSE): 0.7527561996655483
R-squared (R2): 0.8327497475736734

Feature Importance:
{'f0': 854.0, 'f1': 437.0, 'f2': 483.0, 'f3': 421.0, 'f4': 720.0, 'f5': 677.0, 'f6': 621.0, 'f7': 122.0}


In [14]:
# Retrieve feature importance from the XGBoost model
feature_importance = xgb_reg.get_booster().get_score(importance_type='weight')

# Display the feature importance summary
print("Feature Importance:")
for feature, importance in feature_importance.items():
    print(f"{feature}: {importance}")


Feature Importance:
f0: 854.0
f1: 437.0
f2: 483.0
f3: 421.0
f4: 720.0
f5: 677.0
f6: 621.0
f7: 122.0


### lazypredict

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyRegressor
import numpy as np


clf = LazyRegressor(verbose=0,predictions=True, ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

 24%|██▍       | 10/42 [00:04<00:14,  2.15it/s]


KeyboardInterrupt: 